##  Decorators

In [21]:
#decorators as function object
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

@decorate1
def hello():
    print("hello, world!")
        
hello()

apply decorate1
hello, world!


In [2]:
#this is what actually happen when u use a declarator
%reset -f 
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

def hello():
    print("hello, world!")

hello = decorate1(hello)
hello()

#time this DEFINE and creates a closure: a class object: u give a function to time_this, it creates a new
#function and returns it. If you do 
    #hello = decorate1(hello)
#steps: decorate1 receives hello, define a closure function(?) and returns it. Then u reassign this to hello
#So now using "hello" u are calling the object created by the decorator

apply decorate1
hello, world!


In [3]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        func()
    return inner

@decorate1
@decorate2
def hello12():
    print("hello, world!")
hello12() # same as hello12 = decorate1(decorate2(hello12))

apply decorate1
apply decorate2
hello, world!


In [6]:
@decorate2
@decorate1
def hello21():
    print("hello, world!") 
hello21() # same as hello21 = decorate2(decorate1(hello21))
#u can have multiple decorator to same function. Order matters

apply decorate2
apply decorate1
hello, world!


###  How to pass arguments to the inner function

In [27]:
#adapted from Fluent Python
import functools
def args_to_string(*args,**kw):
    arg_str = []
    if args:
        arg_str.append(','.join(str(arg) for arg in args))
    if kw:
        arg_str.append(', '.join(('{0}={1}'.format(k,v) for k,v in kw.items())))
    return ','.join(a for a in arg_str)

import time
def time_this(func):
    def decorated(*args,**kw): #may accept positional and keyward arguments
        t0 = time.perf_counter()
        result = func(*args,**kw)
        t1 = time.perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args,**kw)
        #print('{0}({1}): [{2}]'.format(name, arg_str,t1-t0))  
        #print('{}({}): [{}]'.format(name, arg_str,t1-t0))   #arg_str is a string containing all the arguments
        print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
        return result
    return decorated

#decorated return the result of the func but print the time

@time_this
def wait(seconds):
    time.sleep(seconds)

@functools.lru_cache() # <-- note () # parametrized decorators:retrieve data from cache
@time_this
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

@time_this
def sum(a,b):
    return a+b

@time_this
def dummy(*args, **kw):
    a = args
    b = kw

wait(0.3)
factorial(10)
sum(4,5)
dummy('pos', 'second', a='a', b='b')

wait(0.3): [0.30043521s]
factorial(1): [0.00000161s]
factorial(2): [0.00009392s]
factorial(3): [0.00088229s]
factorial(4): [0.00101830s]
factorial(5): [0.00114757s]
factorial(6): [0.00121242s]
factorial(7): [0.00127111s]
factorial(8): [0.00133997s]
factorial(9): [0.00140367s]
factorial(10): [0.00147117s]
sum(4,5): [0.00000230s]
dummy(pos,second,a=a, b=b): [0.00000358s]


In [28]:
factorial(10)
#only the first execution a time is shown:time_this called only once
factorial(4)
factorial(11)

factorial(11): [0.00000280s]


39916800

In [29]:
#PARAMETRIZED DECORATOR
import time
def parametrized_time_this(check=True): #check=if u want to use the decorator
    def decorator(func): #actual decorator
        if not check:
            return func #if check false, u call func the function itself, so ur not decorating
        def decorated(*args,**kw): #argumento for the decorator
            t0 = time.perf_counter()
            result = func(*args,**kw)
            t1 = time.perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args,**kw)
            print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
            return result
        return decorated
    return decorator # <-- returns the actual decorator
    
@parametrized_time_this(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40146626s]


### Decorators as function objects

In [30]:
import time
class TimeThis():
    def __init__(self, func):           # <--
        self._func = func               # <--
        
    def __call__(self, *args, **kw):
        t0 = time.perf_counter()
        result = self._func(*args,**kw) # <--
        t1 = time.perf_counter()
        name = self._func.__name__      # <--
        arg_str = args_to_string(*args,**kw)
        print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
        return result

@TimeThis
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40084209s]


In [31]:
def func(*args,**kw):
    pass

func = TimeThis(func)
func(0.5)

func(0.5): [0.00000175s]


In [32]:
class ParametrizedTimeThis():
    def __init__(self, check=True):
        self.check = check
    def __call__(self,func):
        if self.check:
            return TimeThis(func) #if I dont use it the output is wrapper(try)
            @TimeThis
            def wrapper(*args,**kwargs):
                return func(*args,**kwargs)
            return wrapper
        return func
        
@ParametrizedTimeThis(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40075710s]


In [33]:
PTT = ParametrizedTimeThis(True)

def dummy(*args,**kw):
    pass

dummy = PTT(dummy)
dummy(0.4)

dummy(0.4): [0.00000189s]
